In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, EfficientNetB0, MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, BatchNormalization, GlobalAveragePooling2D

In [4]:
train_dir = 'New Plant Diseases Dataset(Augmented)/train'
valid_dir = 'New Plant Diseases Dataset(Augmented)/valid'

img_height = 224
img_width = 224
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  
)

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

test_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

num_classes = train_generator.num_classes

callbacks = [
    EarlyStopping(patience=10, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.2, patience=5)
]

Found 56251 images belonging to 38 classes.
Found 14044 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.


In [3]:
# 1. Simple CNN Model
def create_simple_cnn():
    model = Sequential([
        Conv2D(32, 3, activation='relu', input_shape=(img_height, img_width, 3)),
        BatchNormalization(),
        MaxPooling2D(),
        
        Conv2D(64, 3, activation='relu'),
        BatchNormalization(),
        MaxPooling2D(),
        
        Conv2D(128, 3, activation='relu'),
        BatchNormalization(),
        MaxPooling2D(),
        
        Flatten(),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

# 2. ResNet50 Model
def create_resnet50():
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

# 3. EfficientNetB0 Model
def create_efficientnet():
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

# 4. MobileNetV2 Model
def create_mobilenet():
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

In [6]:
def train_and_evaluate_model(model, model_name):
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy',
                tf.keras.metrics.Precision(name='precision'),
                tf.keras.metrics.Recall(name='recall')]
    )
    
    history = model.fit(
        train_generator,
        validation_data=valid_generator, 
        epochs=20,
        callbacks=callbacks
    )
    
    last_epoch = history.history
    train_acc = last_epoch['accuracy'][-1]
    train_precision = last_epoch['precision'][-1]
    train_recall = last_epoch['recall'][-1]
    
    test_results = model.evaluate(
        test_generator,
        verbose=1
    )
    
    test_acc = test_results[1]
    test_precision = test_results[2]
    test_recall = test_results[3]
    
    return {
        'model': model_name,
        'train_acc': train_acc,
        'test_acc': test_acc,
        'train_precision': train_precision,
        'test_precision': test_precision,
        'train_recall': train_recall,
        'test_recall': test_recall
    }

In [6]:
models = {
    #'Simple CNN': create_simple_cnn(),
    'ResNet50': create_resnet50(),
    'EfficientNetB0': create_efficientnet(),
    'MobileNetV2': create_mobilenet()
}

results = []
for name, model in models.items():
    print(f"Training {name}...")
    result = train_and_evaluate_model(model, name)
    results.append(result)
    # save model
    model.save(f'{name}.keras')

Training ResNet50...
Epoch 1/20


/home/ziad/miniconda3/envs/tf-gpu/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1729948087.875341    1311 service.cc:146] XLA service 0x7f98ac002600 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729948087.875385    1311 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-10-26 16:08:08.396636: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-26 16:08:10.600271: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN versi

 968/1758 ━━━━━━━━━━━━━━━━━━━━ 2:33 194ms/step - accuracy: 0.6919 - loss: 1.1653 - precision: 0.8060 - recall: 0.6144

2024-10-26 16:11:48.309847: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_15551', 176 bytes spill stores, 176 bytes spill loads

2024-10-26 16:11:48.454402: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_15551', 28 bytes spill stores, 28 bytes spill loads

2024-10-26 16:11:48.515806: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_17630', 12 bytes spill stores, 12 bytes spill loads

2024-10-26 16:11:48.716379: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_17630', 12 bytes spill stores, 12 bytes spill loads

2024-10-26 16:11:48.867347: I external/local_xla/xla/s

1758/1758 ━━━━━━━━━━━━━━━━━━━━ 451s 230ms/step - accuracy: 0.7670 - loss: 0.8604 - precision: 0.8528 - recall: 0.7084 - val_accuracy: 0.4909 - val_loss: 2.7336 - val_precision: 0.5394 - val_recall: 0.4649 - learning_rate: 0.0010
Epoch 2/20
1758/1758 ━━━━━━━━━━━━━━━━━━━━ 369s 210ms/step - accuracy: 0.9481 - loss: 0.1608 - precision: 0.9570 - recall: 0.9402 - val_accuracy: 0.7254 - val_loss: 1.0443 - val_precision: 0.7704 - val_recall: 0.6964 - learning_rate: 0.0010
Epoch 3/20
1758/1758 ━━━━━━━━━━━━━━━━━━━━ 369s 210ms/step - accuracy: 0.9638 - loss: 0.1110 - precision: 0.9695 - recall: 0.9592 - val_accuracy: 0.5415 - val_loss: 2.3096 - val_precision: 0.5810 - val_recall: 0.5162 - learning_rate: 0.0010
Epoch 4/20
1758/1758 ━━━━━━━━━━━━━━━━━━━━ 368s 209ms/step - accuracy: 0.9715 - loss: 0.0878 - precision: 0.9749 - recall: 0.9683 - val_accuracy: 0.9105 - val_loss: 0.2891 - val_precision: 0.9263 - val_recall: 0.9013 - learning_rate: 0.0010
Epoch 5/20
1758/1758 ━━━━━━━━━━━━━━━━━━━━ 367s 209m

...

 754/1758 ━━━━━━━━━━━━━━━━━━━━ 1:35 95ms/step - accuracy: 0.7989 - loss: 0.7896 - precision: 0.8750 - recall: 0.7601

2024-10-26 18:14:54.212676: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_19979', 28 bytes spill stores, 28 bytes spill loads

2024-10-26 18:14:54.284217: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_22070', 12 bytes spill stores, 12 bytes spill loads

2024-10-26 18:14:54.345063: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_19979', 172 bytes spill stores, 172 bytes spill loads

2024-10-26 18:14:54.694722: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_22072', 32 bytes spill stores, 32 bytes spill loads

2024-10-26 18:14:54.892309: I external/local_xla/xla/s

1758/1758 ━━━━━━━━━━━━━━━━━━━━ 302s 136ms/step - accuracy: 0.8667 - loss: 0.5044 - precision: 0.9135 - recall: 0.8421 - val_accuracy: 0.9332 - val_loss: 0.2700 - val_precision: 0.9403 - val_recall: 0.9281 - learning_rate: 0.0010
Epoch 2/20
1758/1758 ━━━━━━━━━━━━━━━━━━━━ 186s 106ms/step - accuracy: 0.9700 - loss: 0.0993 - precision: 0.9739 - recall: 0.9666 - val_accuracy: 0.9715 - val_loss: 0.0967 - val_precision: 0.9733 - val_recall: 0.9700 - learning_rate: 0.0010
Epoch 3/20
1758/1758 ━━━━━━━━━━━━━━━━━━━━ 186s 106ms/step - accuracy: 0.9770 - loss: 0.0740 - precision: 0.9795 - recall: 0.9753 - val_accuracy: 0.9557 - val_loss: 0.1627 - val_precision: 0.9597 - val_recall: 0.9520 - learning_rate: 0.0010
Epoch 4/20
1758/1758 ━━━━━━━━━━━━━━━━━━━━ 186s 106ms/step - accuracy: 0.9765 - loss: 0.0772 - precision: 0.9785 - recall: 0.9740 - val_accuracy: 0.9725 - val_loss: 0.1079 - val_precision: 0.9745 - val_recall: 0.9709 - learning_rate: 0.0010
Epoch 5/20
1758/1758 ━━━━━━━━━━━━━━━━━━━━ 185s 105m

2024-10-26 19:16:45.603089: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2098', 28 bytes spill stores, 28 bytes spill loads

2024-10-26 19:16:45.678005: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2098', 156 bytes spill stores, 156 bytes spill loads



439/439 ━━━━━━━━━━━━━━━━━━━━ 26s 58ms/step - accuracy: 0.9962 - loss: 0.0115 - precision: 0.9963 - recall: 0.9962
Training MobileNetV2...
Epoch 1/20
1758/1758 ━━━━━━━━━━━━━━━━━━━━ 214s 102ms/step - accuracy: 0.8152 - loss: 0.6602 - precision: 0.8754 - recall: 0.7762 - val_accuracy: 0.4462 - val_loss: 4.3426 - val_precision: 0.4607 - val_recall: 0.4372 - learning_rate: 0.0010
Epoch 2/20
1758/1758 ━━━━━━━━━━━━━━━━━━━━ 158s 90ms/step - accuracy: 0.9493 - loss: 0.1630 - precision: 0.9581 - recall: 0.9428 - val_accuracy: 0.3108 - val_loss: 7.2092 - val_precision: 0.3182 - val_recall: 0.3051 - learning_rate: 0.0010
Epoch 3/20
1758/1758 ━━━━━━━━━━━━━━━━━━━━ 156s 89ms/step - accuracy: 0.9612 - loss: 0.1235 - precision: 0.9664 - recall: 0.9571 - val_accuracy: 0.3335 - val_loss: 7.7118 - val_precision: 0.3388 - val_recall: 0.3266 - learning_rate: 0.0010
Epoch 4/20
1758/1758 ━━━━━━━━━━━━━━━━━━━━ 158s 90ms/step - accuracy: 0.9669 - loss: 0.1073 - precision: 0.9716 - recall: 0.9638 - val_accuracy: 

In [7]:
print("\nResults Table:")
print("Model | Train Acc | Test Acc | Train Precision | Test Precision | Train Recall | Test Recall")
print("-" * 85)
for r in results:
    print(f"{r['model']} | {r['train_acc']:.3f} | {r['test_acc']:.3f} | {r['train_precision']:.3f} | {r['test_precision']:.3f} | {r['train_recall']:.3f} | {r['test_recall']:.3f}")


Results Table:
Model | Train Acc | Test Acc | Train Precision | Test Precision | Train Recall | Test Recall
-------------------------------------------------------------------------------------
ResNet50 | 0.994 | 0.980 | 0.994 | 0.981 | 0.994 | 0.979
EfficientNetB0 | 1.000 | 0.997 | 1.000 | 0.997 | 1.000 | 0.997
MobileNetV2 | 0.985 | 0.435 | 0.987 | 0.450 | 0.984 | 0.428


In [12]:
""""
the session is closed after the first model was trained, so there is the result of cnn
"""

# print("\nResults Table:")
# print("Model | Train Acc | Test Acc | Train Precision | Test Precision | Train Recall | Test Recall")
# print("-" * 85)
# for r in results:
#     print(f"{r['model']} | {r['train_acc']:.3f} | {r['test_acc']:.3f} | {r['train_precision']:.3f} | {r['test_precision']:.3f} | {r['train_recall']:.3f} | {r['test_recall']:.3f}")


Results Table:
Model | Train Acc | Test Acc | Train Precision | Test Precision | Train Recall | Test Recall
-------------------------------------------------------------------------------------
Simple CNN | 0.999 | 0.974 | 0.999 | 0.975 | 0.999 | 0.973


In [5]:
# train densenet21 model

from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input

def create_densenet121():
    base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

densenet121 = create_densenet121()
densenet121.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy',
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall')]
)

I0000 00:00:1729978334.855435    7505 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729978334.992396    7505 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729978334.993307    7505 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729978334.996689    7505 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729978334.997552    7505 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 14s 0us/step


In [7]:
results = []

print(f"Training DenseNet121...")
result = train_and_evaluate_model(densenet121, 'DenseNet121')
results.append(result)
# save model
densenet121.save('DenseNet121.keras')

Training DenseNet121...
Epoch 1/20


/home/ziad/miniconda3/envs/tf-gpu/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1729978944.968185    8035 service.cc:146] XLA service 0x7fd5d0004af0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729978944.969266    8035 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-10-27 00:42:26.167482: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-27 00:42:31.075429: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN versi

 109/1758 ━━━━━━━━━━━━━━━━━━━━ 5:21 195ms/step - accuracy: 0.4259 - loss: 2.3008 - precision: 0.6367 - recall: 0.3052

2024-10-27 00:44:24.398738: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36118', 12 bytes spill stores, 12 bytes spill loads

2024-10-27 00:44:24.540462: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36118', 12 bytes spill stores, 12 bytes spill loads

2024-10-27 00:44:24.952064: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36120', 32 bytes spill stores, 32 bytes spill loads

2024-10-27 00:44:25.602598: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36120', 32 bytes spill stores, 32 bytes spill loads

2024-10-27 00:44:25.789094: I external/local_xla/xla/str

1758/1758 ━━━━━━━━━━━━━━━━━━━━ 605s 274ms/step - accuracy: 0.8252 - loss: 0.6245 - precision: 0.8913 - recall: 0.7834 - val_accuracy: 0.8787 - val_loss: 0.4277 - val_precision: 0.8930 - val_recall: 0.8698 - learning_rate: 0.0010
Epoch 2/20
1758/1758 ━━━━━━━━━━━━━━━━━━━━ 369s 210ms/step - accuracy: 0.9608 - loss: 0.1262 - precision: 0.9672 - recall: 0.9551 - val_accuracy: 0.8079 - val_loss: 0.8056 - val_precision: 0.8317 - val_recall: 0.7933 - learning_rate: 0.0010
Epoch 3/20
1758/1758 ━━━━━━━━━━━━━━━━━━━━ 384s 218ms/step - accuracy: 0.9704 - loss: 0.0932 - precision: 0.9747 - recall: 0.9676 - val_accuracy: 0.9067 - val_loss: 0.3216 - val_precision: 0.9136 - val_recall: 0.9014 - learning_rate: 0.0010
Epoch 4/20
1758/1758 ━━━━━━━━━━━━━━━━━━━━ 383s 218ms/step - accuracy: 0.9751 - loss: 0.0772 - precision: 0.9787 - recall: 0.9727 - val_accuracy: 0.9643 - val_loss: 0.1264 - val_precision: 0.9673 - val_recall: 0.9620 - learning_rate: 0.0010
Epoch 5/20
1758/1758 ━━━━━━━━━━━━━━━━━━━━ 374s 213m

2024-10-27 02:51:33.315375: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_3340', 172 bytes spill stores, 172 bytes spill loads



439/439 ━━━━━━━━━━━━━━━━━━━━ 46s 104ms/step - accuracy: 0.9953 - loss: 0.0154 - precision: 0.9954 - recall: 0.9949


In [8]:
print("\nResults Table:")
print("Model | Train Acc | Test Acc | Train Precision | Test Precision | Train Recall | Test Recall")
print("-" * 85)
for r in results:
    print(f"{r['model']} | {r['train_acc']:.3f} | {r['test_acc']:.3f} | {r['train_precision']:.3f} | {r['test_precision']:.3f} | {r['train_recall']:.3f} | {r['test_recall']:.3f}")



Results Table:
Model | Train Acc | Test Acc | Train Precision | Test Precision | Train Recall | Test Recall
-------------------------------------------------------------------------------------
DenseNet121 | 0.999 | 0.997 | 0.999 | 0.997 | 0.999 | 0.996
